In [38]:
import os
import streamlit as st
import pickle
import time
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import pipeline, AutoModelForSeq2SeqLM, AutoTokenizer
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from langchain.document_loaders import UnstructuredURLLoader

In [39]:
# Load Hugging Face LLM (Alternative to OpenAI GPT)
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
generator = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

Device set to use cpu


In [40]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")


In [41]:
# Load web data
loaders = UnstructuredURLLoader(urls=[
    "https://www.moneycontrol.com/news/business/markets/wall-street-rises-as-tesla-soars-on-ai-optimism-11351111.html",
    "https://www.moneycontrol.com/news/business/tata-motors-launches-punch-icng-price-starts-at-rs-7-1-lakh-11098751.html"
])
data = loaders.load() 

In [43]:
# Split text into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

# As data is of type documents we can directly use split_documents over split_text in order to get the chunks.
docs = text_splitter.split_documents(data)


In [46]:
embeddings = np.array([embedding_model.encode(doc.page_content) for doc in docs])


In [47]:
# Create FAISS index
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)


In [48]:
# Save FAISS index
file_path = "vector_index.pkl"
with open(file_path, "wb") as f:
    pickle.dump(index, f)


In [52]:
# Retrieve similar embeddings and call LLM for final answer
def query_llm(question, top_k=3):
    question_embedding = embedding_model.encode(question).reshape(1, -1)
    D, I = index.search(question_embedding, k=top_k)  # Retrieve top-k relevant matches
    relevant_texts = " ".join([docs[i].page_content for i in I[0]])  # Combine top-k relevant documents
    
    prompt = f"Context: {relevant_texts}\n\nQuestion: {question} Answer:"
    response = generator(prompt, max_length=200, do_sample=True)
    return response[0]['generated_text']

In [53]:
# Example Query
query = "What is the price of Tiago iCNG?"
answer = query_llm(query)
print("Answer:", answer)


Token indices sequence length is longer than the specified maximum sequence length for this model (702 > 512). Running this sequence through the model will result in indexing errors


Answer: Rs 6.55 lakh and Rs 8.1 lakh
